Group: Austin Wang, Joe Higgins, Lawrence Moore

# Question 1

# SOCP

In [1]:
# Import necessary modules
import cvxpy as cvx
import numpy as np

In [2]:
def generate_points(num, dim):
    return np.matrix(4 * np.random.random((num, dim)) - 2)

In [121]:
num_sensors = 10
num_anchors = 4
dim = 2
range_mult = 1

anchors = generate_points(num_anchors, dim) * range_mult
sensors = generate_points(num_sensors, dim)

d_sa = list(map(lambda s: list(map(lambda a: np.linalg.norm(a - s), anchors)), sensors))

d_ss = list(map(lambda s1: 
        list(map(lambda s2: np.linalg.norm(s1 - s2), sensors))
    , sensors))

In [4]:
x = cvx.Variable(num_sensors, dim)
objective = cvx.Minimize(0)

In [5]:
constraints  = []
for i in range(num_sensors):
    x_i = x[i, :]
    for j in range(num_anchors):
        constraints.append(cvx.norm(x_i - anchors[j]) <= d_sa[i][j])
    for j in range(num_sensors):
        if i < j:
            constraints.append(cvx.norm(x_i - sensors[j]) <= d_ss[i][j])


In [6]:
print(sensors)

[[-0.82963175  1.49491793]
 [-1.6397261  -0.88638492]
 [-0.57167572 -1.90004467]
 [-0.57647951 -1.41087409]
 [ 0.12781571 -0.32245923]
 [-1.03637545 -0.24204694]
 [-1.55879562  1.60678321]
 [ 1.34062095  1.62617786]
 [ 0.88287445 -1.71321207]
 [ 1.62936614  0.73302684]]


In [7]:
print(anchors)

[[  71.26364541   31.19531126]
 [  79.36622871  132.69536359]
 [ -95.36945673  116.07136195]
 [ -80.2814842  -168.8118134 ]]


In [8]:
prob = cvx.Problem(objective, constraints)

# The optimal objective is returned by prob.solve().
result = prob.solve(solver = 'MOSEK')
# The optimal value for x is stored in x.value.
print(x.value)

[[-0.82963175  1.49491793]
 [-1.6397261  -0.88638492]
 [-0.57167572 -1.90004467]
 [-0.57647951 -1.41087409]
 [ 0.12781571 -0.32245923]
 [-1.03637545 -0.24204694]
 [-1.55879562  1.60678321]
 [ 1.34062095  1.62617786]
 [ 0.88287445 -1.71321207]
 [ 1.62936614  0.73302684]]


In [9]:
print(abs(x.value-sensors)<= 10**-5)

[[ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]]


# SDP

In [10]:
Z = cvx.Semidef(num_sensors + dim)

In [11]:
def sum_elem_product(A,B):
    return cvx.sum_entries(cvx.mul_elemwise(A, B))

In [12]:
#Make first set of constraint matrices (look like identity)
def enforce_id(dim, num_sensors):
    matrices = []
    rhs = []
    for i in range(dim):
        new_matrix = np.zeros((dim+num_sensors, dim+num_sensors))
        new_matrix[i,i] = 1
        matrices.append(new_matrix)
        rhs.append(1)
        
    return (matrices, rhs)

#Make second set of constraint matrices (symmetric holders) 
def enforce_id2(dim, num_sensors):
    matrices = []
    rhs = []
    for i in range(dim):
        for j in range(dim):
            new_matrix = np.identity(dim)
            if(j > i):
                new_matrix[i,j] = 1
                new_matrix[j,i] = 1
                big_matrix = np.zeros((dim+num_sensors, dim+num_sensors))
                big_matrix[0:dim,0:dim] = new_matrix
                matrices.append(big_matrix)
                rhs.append(dim)
                
    return (matrices, rhs)


#Make third set of constraint matrices (anchors to sensors)
def sensor_constraints(dim, num_sensors):
    matrices = []
    rhs = []
    zero_vec_dim = np.zeros(dim)
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            zero_vec_num_s = np.zeros(num_sensors)
            zero_vec_num_s[i] = 1
            zero_vec_num_s[j] = -1
            
            new_vec = np.matrix(np.append(zero_vec_dim, zero_vec_num_s))
            
            new_matrix = np.dot(np.transpose(new_vec), new_vec)
            
            matrices.append(new_matrix)
            rhs.append(d_ss[i][j]**2)

    return (matrices, rhs)

#Make fourth set of constraint matrices (sensors to sensors)
def anchor_constraints(num_anchors, num_sensors):
    matrices = []
    rhs = []
    for i in range(num_anchors):
        for j in range(num_sensors):
            zero_vec_num_s = np.zeros(num_sensors)
            zero_vec_num_s[j] = -1

            new_vec = np.append(np.array(anchors[i,:]), np.array(zero_vec_num_s))
            new_vec = np.matrix(new_vec)
            new_matrix = np.dot(np.transpose(new_vec), new_vec)
            matrices.append(new_matrix)
            rhs.append(d_sa[j][i]**2)

    return (matrices, rhs)

A = enforce_id(dim, num_sensors)
B = enforce_id2(dim, num_sensors)
C = sensor_constraints(dim, num_sensors)
D = anchor_constraints(num_anchors, num_sensors)

In [13]:
constraints = []
for id_constraint, rhs in zip(A[0], A[1]):
    constraints.append(sum_elem_product(id_constraint, Z) == rhs)
for id_constraint2, rhs in zip(B[0], B[1]):
    constraints.append(sum_elem_product(id_constraint2, Z) == rhs)
for sensor_constraint, rhs in zip(C[0], C[1]):
    constraints.append(sum_elem_product(sensor_constraint, Z) == rhs)
for anchor_constraint, rhs in zip(D[0], D[1]):
    constraints.append(sum_elem_product(anchor_constraint, Z) == rhs)

In [14]:
prob = cvx.Problem(objective, constraints)
result = prob.solve(solver = 'MOSEK')

In [15]:
print("Real sensors: ")
print(sensors)
print("Generated Sensors: ")
generated = np.transpose(Z[0:dim, dim:dim+num_sensors].value)
print(generated)

Real sensors: 
[[-0.82963175  1.49491793]
 [-1.6397261  -0.88638492]
 [-0.57167572 -1.90004467]
 [-0.57647951 -1.41087409]
 [ 0.12781571 -0.32245923]
 [-1.03637545 -0.24204694]
 [-1.55879562  1.60678321]
 [ 1.34062095  1.62617786]
 [ 0.88287445 -1.71321207]
 [ 1.62936614  0.73302684]]
Generated Sensors: 
[[-0.82963175  1.49491793]
 [-1.6397261  -0.88638492]
 [-0.57167572 -1.90004467]
 [-0.57647951 -1.41087409]
 [ 0.12781571 -0.32245923]
 [-1.03637545 -0.24204694]
 [-1.55879562  1.60678321]
 [ 1.34062095  1.62617786]
 [ 0.88287445 -1.71321207]
 [ 1.62936614  0.73302684]]


In [16]:
def super_quick_func(real, generated):
    for_real_do_results = []
    for real_sens, real_gen in zip(real, generated):
        for_real_do_results.append(np.linalg.norm(real_sens - real_gen) < 10**(-6))
    return for_real_do_results


In [17]:
super_quick_func(sensors, generated)

[True, True, True, True, True, True, True, True, True, True]

Yo, anchors is huge == it'll localize.

### Part 3 

In [18]:
def grad(X):
    sensor_distance_sum = np.zeros((1,X.shape[1]))
    anchor_distance_sum = np.zeros((1,X.shape[1]))
    gradient = np.zeros(X.shape)

    for i, sensor_i in enumerate(X):
        for j, sensor_j in enumerate(X):
            if(i != j):
                sensor_distance_sum += (np.linalg.norm(sensor_i - sensor_j)**2 - \
                                        np.linalg.norm(sensors[i,:] - sensors[j,:])**2) * \
                                       (sensor_i - sensor_j)

        for k, anchor_k in enumerate(anchors):
            anchor_distance_sum += (np.linalg.norm(anchor_k - sensor_i)**2 - \
                                    np.linalg.norm(anchors[k,:] - sensors[i,:])**2) * \
                                   (sensor_i - anchor_k)

        gradient[i,:] = 8*sensor_distance_sum + 4*anchor_distance_sum

    return gradient

In [33]:
check = 1000
max_iter = 10000
k = 0

# Initial sensors guess
sensors_0 = generate_points(num_sensors, dim)
sensors_k = sensors_0

# D iteration
alpha = .0000001
while check > 10**-8 and k < max_iter:
    sensors_k1 = sensors_k - alpha * grad(sensors_k)
    check = np.linalg.norm(sensors_k1 - sensors_k)
    sensors_k = sensors_k1
    k = k+1
    

In [31]:
sensors

matrix([[-0.82963175,  1.49491793],
        [-1.6397261 , -0.88638492],
        [-0.57167572, -1.90004467],
        [-0.57647951, -1.41087409],
        [ 0.12781571, -0.32245923],
        [-1.03637545, -0.24204694],
        [-1.55879562,  1.60678321],
        [ 1.34062095,  1.62617786],
        [ 0.88287445, -1.71321207],
        [ 1.62936614,  0.73302684]])

In [34]:
sensors_k

matrix([[-0.82963175,  1.49491793],
        [-1.6397261 , -0.88638492],
        [-0.57167572, -1.90004467],
        [-0.57647952, -1.41087409],
        [ 0.12781572, -0.32245924],
        [-1.03637549, -0.24204693],
        [-1.55879554,  1.60678318],
        [ 1.34062092,  1.62617787],
        [ 0.88287422, -1.71321199],
        [ 1.62936685,  0.73302657]])

### Question 2

In [122]:
#order of indexing: (1,2), (1,3), (1,4)... (9,10)
delta_p_ss = cvx.Variable((int) (num_sensors*(num_sensors-1)/2))
delta_pp_ss = cvx.Variable((int) (num_sensors*(num_sensors-1)/2))
                           
#order of indexing: (anchor, sensor), primary sort on anchors
delta_p_sa = cvx.Variable(num_anchors * num_sensors)
delta_pp_sa = cvx.Variable(num_anchors * num_sensors)

In [123]:
constraints = []

constraints.append(delta_p_ss >= 0)
constraints.append(delta_pp_ss >= 0)
constraints.append(delta_p_sa >= 0)
constraints.append(delta_pp_sa >= 0)

for id_constraint, rhs in zip(A[0], A[1]):
    constraints.append(sum_elem_product(id_constraint, Z) == rhs)
for id_constraint2, rhs in zip(B[0], B[1]):
    constraints.append(sum_elem_product(id_constraint2, Z) == rhs)
for sensor_constraint, rhs, dp, dpp in zip(C[0], C[1], delta_p_ss, delta_pp_ss):
    constraints.append(sum_elem_product(sensor_constraint, Z) + \
                       dp - dpp == rhs)
for anchor_constraint, rhs, dp, dpp in zip(D[0], D[1], delta_p_sa, delta_pp_sa):
    constraints.append(sum_elem_product(anchor_constraint, Z) + \
                       dp - dpp == rhs)


In [124]:
objective = cvx.Minimize(cvx.sum_entries(delta_p_ss) + \
                         cvx.sum_entries(delta_pp_ss) + \
                         cvx.sum_entries(delta_p_sa) + \
                         cvx.sum_entries(delta_pp_sa))

In [125]:
prob = cvx.Problem(objective, constraints)
result = prob.solve(solver = 'MOSEK')

In [126]:
print("Real sensors: ")
print(sensors)
print("Generated Sensors: ")
generated = np.transpose(Z[0:dim, dim:dim+num_sensors].value)
print(generated)

Real sensors: 
[[ 1.67195583 -1.36603783]
 [ 1.26190505 -0.08582429]
 [ 1.23018811 -1.77928546]
 [ 1.17257755  0.97413599]
 [-0.12593672 -1.75731458]
 [-0.42770344  0.04298314]
 [ 1.96740319 -1.32359296]
 [-0.16986075  0.22900364]
 [-0.85774339 -1.88235852]
 [-1.02585377  0.78144002]]
Generated Sensors: 
[[-0.82963175  1.49491793]
 [-1.6397261  -0.88638492]
 [-0.57167572 -1.90004467]
 [-0.57647951 -1.41087409]
 [ 0.12781571 -0.32245923]
 [-1.03637545 -0.24204694]
 [-1.55879562  1.60678321]
 [ 1.34062094  1.62617786]
 [ 0.88287445 -1.71321207]
 [ 1.62936614  0.73302684]]


In [131]:
check = 1000
max_iter = 10000
k = 0

# Initial sensors guess
sensors_0 = generated
sensors_k = sensors_0

# D iteration
alpha = .000001
while check > 10**-8 and k < max_iter:
    sensors_k1 = sensors_k - alpha * grad(sensors_k)
    check = np.linalg.norm(sensors_k1 - sensors_k)
    sensors_k = sensors_k1
    print(grad(sensors_k))
    k = k+1
    

[[   70.75020499   484.47374548]
 [ -459.74595907   -77.59271889]
 [ -501.6645698   -894.45488534]
 [ -582.96393607 -1448.31672101]
 [ -622.42893109 -1431.7425938 ]
 [ -976.0552325  -1582.90018343]
 [-1151.73133721  -944.98100064]
 [ -175.52540991   282.73023102]
 [  -25.81200213  -300.95223862]
 [  315.74693766   -36.41718282]]
[[   70.80636343   483.44996543]
 [ -459.30119703   -78.4484428 ]
 [ -500.9257711   -894.22572005]
 [ -581.85129951 -1446.8965675 ]
 [ -621.30468044 -1429.99832672]
 [ -974.34034012 -1580.52127613]
 [-1149.46132099  -942.82974818]
 [ -174.22317348   283.36320263]
 [  -24.71317171  -299.89859131]
 [  315.71951201   -35.96787866]]
[[   70.86286716   482.42914399]
 [ -458.85598555   -79.30060243]
 [ -500.18683541  -893.99384739]
 [ -580.73939672 -1445.47638849]
 [ -620.18130209 -1428.25513162]
 [ -972.6279478  -1578.14545183]
 [-1147.1951675   -940.68215384]
 [ -172.92433237   283.99410356]
 [  -23.61738267  -298.84785216]
 [  315.69143779   -35.52007851]]
[[   70

[[   73.86940545   437.60517852]
 [ -437.49577227  -115.56331636]
 [ -465.38805595  -880.31240569]
 [ -529.36504087 -1378.87096604]
 [ -568.41668255 -1347.64126057]
 [ -894.92638255 -1469.93938707]
 [-1044.92109611  -843.80668669]
 [ -115.54075957   311.39257268]
 [   24.61095577  -252.61353823]
 [  313.70370612   -16.08735655]]
[[   73.94006937   436.71489649]
 [ -437.0334751   -116.25895393]
 [ -464.647884    -879.96752362]
 [ -528.29188349 -1377.46022756]
 [ -567.33824035 -1345.95610966]
 [ -893.33142711 -1467.71015356]
 [-1042.83258815  -841.8304596 ]
 [ -114.3942782    311.92929783]
 [   25.57066943  -251.6941516 ]
 [  313.64847381   -15.70653273]]
[[   74.0109822    435.82711014]
 [ -436.57090821  -116.95161547]
 [ -463.90778676  -879.62061403]
 [ -527.21959556 -1376.04988375]
 [ -566.26078696 -1344.27228532]
 [ -891.73885972 -1465.48393543]
 [-1040.74762136  -839.85770836]
 [ -113.25075857   312.46416886]
 [   26.52774589  -250.7773347 ]
 [  313.59275276   -15.32700044]]
[[   74

 [  309.13506972     6.45356812]]
[[   79.07837651   383.08866998]
 [ -406.06914932  -156.08366792]
 [ -416.12154087  -853.27147584]
 [ -459.47066162 -1285.53061405]
 [ -498.41362164 -1237.83328463]
 [ -793.21511685 -1327.14316457]
 [ -912.51767367  -718.8823951 ]
 [  -44.89220944   343.45881989]
 [   83.43519725  -196.38970295]
 [  309.0533037      6.75731135]]
[[   79.16231642   382.34817744]
 [ -405.5952306   -156.60218384]
 [ -415.39349303  -852.81544608]
 [ -458.45942122 -1284.15990858]
 [ -497.40424393 -1236.24414695]
 [ -791.7741115  -1325.11095531]
 [ -910.6525316   -717.12962096]
 [  -43.92675278   343.88072282]
 [   84.23432215  -195.62849847]
 [  308.97121676     7.06000617]]
[[   79.24640803   381.60967469]
 [ -405.12122704  -157.11837124]
 [ -414.66572051  -852.35809934]
 [ -457.44914673 -1282.78997715]
 [ -496.39592606 -1234.65652647]
 [ -790.33530622 -1323.08159961]
 [ -908.79051523  -715.38003166]
 [  -42.96374532   344.30105255]
 [   85.03128303  -194.86944957]
 [  308

 [  296.38665739    38.42605678]]
[[   90.99303773   300.87215937]
 [ -343.9826864   -206.95633867]
 [ -323.78156638  -785.16250856]
 [ -335.37593335 -1113.77965166]
 [ -375.23805545 -1042.96549544]
 [ -622.0635307  -1084.11526428]
 [ -692.72140827  -514.71160574]
 [   63.07800814   386.71529873]
 [  171.84237413  -113.07152066]
 [  296.27828137    38.61743057]]
[[   91.08864353   300.34412016]
 [ -343.513481    -207.23191582]
 [ -323.10442426  -784.59691652]
 [ -334.49376252 -1112.5362877 ]
 [ -374.36722267 -1041.5824225 ]
 [ -620.88478399 -1082.43072893]
 [ -691.21821692  -513.33709426]
 [   63.77570345   386.96260809]
 [  172.40646889  -112.54837647]
 [  296.16980009    38.8081156 ]]
[[   91.18428838   299.81738474]
 [ -343.04440808  -207.50604884]
 [ -322.42775108  -784.03088321]
 [ -333.61256785 -1111.29402733]
 [ -373.49741863 -1040.20092969]
 [ -619.70783801 -1080.74863502]
 [ -689.7174453   -511.9651714 ]
 [   64.47171921   387.2088081 ]
 [  172.96910025  -112.02674071]
 [  296

 [ 266.72718108   70.63584294]]
[[ 116.43648725  196.48966516]
 [-226.6262299  -242.4307626 ]
 [-161.8484163  -630.61440963]
 [-133.47024597 -823.98737144]
 [-177.50858295 -728.84619118]
 [-364.09075332 -713.39047868]
 [-365.92899345 -228.44879465]
 [ 201.6619792   421.4983227 ]
 [ 281.29688644  -16.36241856]
 [ 266.61936459   70.70445621]]
[[ 116.52964498  196.20006428]
 [-226.21566121 -242.46140239]
 [-161.30455258 -630.04303539]
 [-132.81892068 -823.04249538]
 [-176.87541284 -727.84565536]
 [-363.29382716 -712.24407568]
 [-364.92377181 -227.62103377]
 [ 202.04839122  421.53875434]
 [ 281.59294123  -16.12202882]
 [ 266.51162012   70.77277777]]
[[ 116.62276552  195.91106157]
 [-225.80535941 -242.49150232]
 [-160.76117922 -629.47190406]
 [-132.1683393  -822.09868618]
 [-176.2429924  -726.84638584]
 [-362.49802096 -711.09932453]
 [-363.91997278 -226.79487042]
 [ 202.43402226  421.57866837]
 [ 281.88833425  -15.88235629]
 [ 266.40394797   70.84080859]]
[[ 116.71584876  195.62265538]
 [-2

...

[[  1.82565857e+02  -1.56281492e-01]
 [  6.01736168e+01  -5.76623170e+01]
 [  1.60747934e+02  -4.69065656e+01]
 [  2.15370002e-01  -1.30956347e+02]
 [ -2.10618919e+01  -3.54692097e+01]
 [ -2.97334276e+02   1.74769578e+01]
 [ -1.98874187e+02   5.43491942e+01]
 [ -2.11037798e+02   4.40586930e+01]
 [ -1.90460610e+02  -6.81891906e+01]
 [ -6.12379824e+01  -2.26052720e+01]]
[[  1.82524806e+02  -1.18453029e-01]
 [  6.00883354e+01  -5.76136750e+01]
 [  1.60598925e+02  -4.68758340e+01]
 [  3.10512974e-02  -1.30917725e+02]
 [ -2.12587123e+01  -3.54230542e+01]
 [ -2.97512527e+02   1.75210782e+01]
 [ -1.98958624e+02   5.43456083e+01]
 [ -2.11051080e+02   4.40442739e+01]
 [ -1.90439016e+02  -6.81744042e+01]
 [ -6.11935611e+01  -2.25919606e+01]]
[[  1.82483753e+02  -8.06221242e-02]
 [  6.00030818e+01  -5.75650355e+01]
 [  1.60449968e+02  -4.68450947e+01]
 [ -1.53135937e-01  -1.30879104e+02]
 [ -2.14553657e+01  -3.53769114e+01]
 [ -2.97690564e+02   1.75651780e+01]
 [ -1.99042822e+02   5.43420043e+01]

[[ 179.35657581    2.7998567 ]
 [  53.60837032  -53.87808199]
 [ 149.28780088  -44.48946372]
 [ -13.76086674 -127.94871053]
 [ -35.90605285  -31.91176759]
 [-310.58994031   20.85217153]
 [-204.74378338   54.01250617]
 [-211.37009376   42.88110078]
 [-188.13214964  -67.01327361]
 [ -57.5467886   -21.56424537]]
[[ 179.31535315    2.83780397]
 [  53.52538282  -53.82973077]
 [ 149.14310081  -44.45824735]
 [ -13.93470478 -127.91027189]
 [ -36.08960345  -31.86678367]
 [-310.7513219    20.89451497]
 [-204.80966636   54.00740876]
 [-211.3652364    42.86528689]
 [-188.09400839  -66.99791207]
 [ -57.49662929  -21.55087166]]
[[ 179.274129      2.87575182]
 [  53.44242633  -53.78138462]
 [ 148.99845927  -44.42702614]
 [ -14.10840567 -127.87183778]
 [ -36.27298101  -31.82181717]
 [-310.91248583   20.93683344]
 [-204.87531376   54.0022908 ]
 [-211.36015193   42.84945436]
 [-188.05566213  -66.98254359]
 [ -57.44640183  -21.53749747]]
[[ 179.23290337    2.91370023]
 [  53.35950089  -53.73304358]
 [ 14

 [ -55.48859564  -21.02899861]]
[[ 177.66541864    4.35591584]
 [  50.23172652  -51.90020362]
 [ 143.40394759  -43.20598808]
 [ -20.77508417 -126.37702088]
 [ -43.28904787  -30.0823214 ]
 [-317.02782955   22.56720773]
 [-207.25251903   53.78636856]
 [-210.98628071   42.21726543]
 [-186.40224871  -66.37785724]
 [ -55.43581594  -21.01561155]]
[[ 177.62414984    4.39386731]
 [  50.15004241  -51.85208813]
 [ 143.26171284  -43.1746014 ]
 [ -20.94324737 -126.33881246]
 [ -43.46545178  -30.03809873]
 [-317.18026697   22.60848396]
 [-207.30879843   53.78040626]
 [-210.97223288   42.2006736 ]
 [-186.35585606  -66.36221882]
 [ -55.38297351  -21.0022243 ]]
[[ 177.58288029    4.43181838]
 [  50.06839091  -51.80397909]
 [ 143.11953988  -43.14321128]
 [ -21.11127088 -126.30061079]
 [ -43.64168013  -29.99389583]
 [-317.33248581   22.64973308]
 [-207.36484508   53.77442232]
 [-210.95796404   42.18406248]
 [-186.30926612  -66.34657385]
 [ -55.33006848  -20.98883687]]
[[ 177.54161003    4.46976903]
 [  

 [ -51.93329471  -20.15872326]]
[[ 174.98221387    6.82089454]
 [  44.99173296  -48.78756437]
 [ 134.29055041  -41.16021807]
 [ -31.4127655  -123.90980207]
 [ -54.38785299  -27.25137362]
 [-326.48110321   25.19161578]
 [-210.42991553   53.35272261]
 [-209.62027926   41.0980477 ]
 [-182.98485358  -65.34812421]
 [ -51.87671612  -20.14533691]]
[[ 174.9409354     6.85877071]
 [  44.9122466   -48.73993696]
 [ 134.15248999  -41.12867972]
 [ -31.57173781 -123.87214114]
 [ -54.55273992  -27.20854981]
 [-326.61931149   25.2310281 ]
 [-210.47122858   53.34530305]
 [-209.59219216   41.08017343]
 [-182.92604236  -65.33207839]
 [ -51.82008347  -20.13195076]]
[[ 174.8996574     6.89664495]
 [  44.83279526  -48.69231813]
 [ 134.01449624  -41.09714016]
 [ -31.73056708 -123.83449034]
 [ -54.71744819  -27.16574929]
 [-326.75730071   25.27041019]
 [-210.51231411   53.33786032]
 [-209.56389439   41.06227905]
 [-182.86704659  -65.31602658]
 [ -51.76339691  -20.11856482]]
[[ 174.85837989    6.93451722]
 [  

KeyboardInterrupt: 

In [132]:
grad(sensors_k)

array([[ 174.44563529,    7.31312679],
       [  43.96115297,  -48.16908684],
       [ 132.50098531,  -40.75013493],
       [ -33.46823611, -123.42101441],
       [ -56.51743615,  -26.69649479],
       [-328.26072472,   25.7016051 ],
       [-210.94926393,   53.25445518],
       [-209.23876158,   40.86410987],
       [-182.20596994,  -65.13906383],
       [ -51.13632348,  -19.97133489]])